# Road Follower - Train Model

In this notebook we will train a neural network to take an input image, and output a set of x, y values corresponding to a target.

We will be using PyTorch deep learning framework to train ResNet18 neural network architecture model for road follower application.

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Download and extract data

Before you start, you should upload the ``road_following_<Date&Time>.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot.

> If you're training on the JetBot you collected data on, you can skip this!

You should then extract this dataset by calling the command below:

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance

Here we create a custom ``torch.utils.data.Dataset`` implementation, which implements the ``__len__`` and ``__getitem__`` functions.  This class
is responsible for loading images and parsing the x, y values from the image filenames.  Because we implement the ``torch.utils.data.Dataset`` class,
we can use all of the torch data utilities :)

We hard coded some transformations (like color jitter) into our dataset.  We made random horizontal flips optional (in case you want to follow a non-symmetric path, like a road
where we need to 'stay right').  If it doesn't matter whether your robot follows some convention, you could enable flips to augment the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re

def get_steering(path):
    """Gets the steering value from the image filename"""
    match = re.search(r'_s(-?\d+\.\d+)', path)
    if match:
        return float(match.group(1))
    else:
        raise ValueError(f"Cannot find steering value in {path}")


def get_left_speed(input_str):
    # 정규 표현식을 사용하여 'l' 뒤에 오는 숫자를 찾기
    match = re.search(r'l(\d+)', input_str)

    if match:
        # 숫자를 추출하여 반환
        return int(match.group(1))
    else:
        # 'l' 다음에 숫자가 없으면 None 반환
        return None


# 테스트 예시
filename1 = 'image_1753187538_s0.00_l100.00_r100.00.png'
filename2 = 'image_174305318740_s-1.00_l100.00_r100.00.png'

print(get_steering(filename1))  # 출력: 0.0
print(get_steering(filename2))  # 출력: -1.0
print(get_left_speed(filename1))  # 출력: 0.0


0.0
-1.0
100


In [ ]:
import re

def get_steering(path):
    """Gets the steering value from the image filename"""
    match = re.search(r'_s(-?\d+\.\d+)', path)
    if match:
        return float(match.group(1))
    else:
        raise ValueError(f"Cannot find steering value in {path}")

def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 112.0) / 112.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 112.0) / 112.0

class XYDataset(torch.utils.data.Dataset):

    def __init__(self, directory, random_hflips=False):
        print(directory)
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.png'))
        print(self.image_paths)
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]

        image = PIL.Image.open(image_path)
        # x = float(get_x(os.path.basename(image_path)))
        # y = float(get_y(os.path.basename(image_path)))

        steering = float(get_steering(os.path.basename(image_path)))
        speed = float(get_left_speed(os.path.basename(image_path)))

        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            steering = -steering

        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        return image, torch.tensor([steering, speed]).float()
dataset = XYDataset('/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation', random_hflips=False)

/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation
['/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053026328_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053027898_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053027101_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053028429_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053028805_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053029877_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053033574_s-3.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053038590_s-2.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simul

In [ ]:
print(dataset.image_paths)

['/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053026328_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053027898_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053027101_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053028429_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053028805_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053029877_s0.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053033574_s-3.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053038590_s-2.00_l100.00_r100.00.png', '/content/drive/MyDrive/E2E_simulation/e2e_dataset_simulation/image_1743053042519_s-2.00_l100.00_r100.00.png', '/cont

### Split dataset into train and test sets
Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches

We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define Neural Network Model

We use ResNet-18 model available on PyTorch TorchVision.

In a process called transfer learning, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.


More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
model = models.resnet18(pretrained=True)

ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

We train for 50 epochs and save best model if the loss is reduced.

In [ ]:
NUM_EPOCHS = 10
BEST_MODEL_PATH = 'best_steering_model_250603_v2.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    print(epoch)
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += float(loss)
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)

    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
    test_loss /= len(test_loader)

    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
torch.save(model.state_dict(), '/content/drive/MyDrive/E2E_simulation/model_250603_v2.pt')


0


<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated

4878.136621, 4144.154785
1


<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated

4482.550879, 3367.005127
2


...

2893.468799, 2562.715820
8


<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated

2647.593213, 2500.679932
9


<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if float(np.random.rand(1)) > 0.5:
<ipython-input-20-093ba4a7622b>:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated

2403.867529, 2257.664062


In [ ]:
torch.save(model, '/content/drive/MyDrive/E2E_simulation/model_250405')

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/driving_capstone_design/code/model_red_temp_250322.pt")

Once the model is trained, it will generate ``best_steering_model_xy.pth`` file which you can use for inferencing in the live demo notebook.

If you trained on a different machine other than JetBot, you'll need to upload this to the JetBot to the ``road_following`` example folder.